In [7]:
# !pip install lightrag
import os

os.environ["GROQ_API_KEY"] = "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"

In [15]:

from dataclasses import dataclass, field

from lightrag.core import Component, Generator, DataClass
from lightrag.components.model_client import GroqAPIClient
from lightrag.components.output_parsers import JsonOutputParser

@dataclass
class QAOutput(DataClass):
    explanation: str = field(
        metadata={"desc": "A brief explanation of the concept in one sentence."}
    )
    example: str = field(metadata={"desc": "An example of the concept in a sentence."})



qa_template = r"""<SYS>
Your are MedSathi a helpful pharmacist. Use the given context to get the knowledge of the medicine and provide the asked information about the medicine Using your own Knowledge.
<OUTPUT_FORMAT>
{{output_format_str}}
</OUTPUT_FORMAT>
</SYS>
User: {{input_str}}
You:"""

class QA(Component):
    def __init__(self):
        super().__init__()

        parser = JsonOutputParser(data_class=QAOutput, return_data_class=True)
        self.generator = Generator(
            model_client=GroqAPIClient(),
            model_kwargs={"model": "llama3-8b-8192"},
            template=qa_template,
            prompt_kwargs={"output_format_str": parser.format_instructions()},
            output_processors=parser,
        )

    def call(self, query: str):
        return self.generator.call({"input_str": query})

    async def acall(self, query: str):
        return await self.generator.acall({"input_str": query})

In [17]:

qa = QA()
# print(qa)


output = qa("Why we use Benzoyl Peroxide Topical")# call
print(output)

GeneratorOutput(data=QAOutput(explanation='Benzoyl peroxide topical is used to treat acne, reducing the number and severity of pimples and blackheads, by killing bacteria that can cause infections and breaking down keratin, a protein that can cause clogs.', example='Dr. Jane prescribed 2.5% benzoyl peroxide cream to her patient, Emma, to treat her moderate acne, instructing her to apply it once or twice daily to specific areas of the face for four to eight weeks to see noticeable improvements.'), error=None, usage=None, raw_response='```\n{\n    "explanation": "Benzoyl peroxide topical is used to treat acne, reducing the number and severity of pimples and blackheads, by killing bacteria that can cause infections and breaking down keratin, a protein that can cause clogs.",\n    "example": "Dr. Jane prescribed 2.5% benzoyl peroxide cream to her patient, Emma, to treat her moderate acne, instructing her to apply it once or twice daily to specific areas of the face for four to eight weeks 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 KB 3.0 MB/s eta 0:00:00


In [27]:
import json

# The given GeneratorOutput string
generator_output_str = """
GeneratorOutput(
    data=QAOutput(
        explanation='Benzoyl peroxide topical is used to treat mild to moderate acne, reduce inflammation and kill bacteria that cause acne.', 
        example='For example, it may be applied directly to the skin to help clear up pimples and reduce redness and inflammation in the affected area.'
    ), 
    error=None, 
    usage=None, 
    raw_response='
    {
        "explanation": "Benzoyl peroxide topical is used to treat mild to moderate acne, reduce inflammation and kill bacteria that cause acne.",
        "example": "For example, it may be applied directly to the skin to help clear up pimples and reduce redness and inflammation in the affected area."
    }
    ', 
    metadata=None
)
"""

# Extract the raw_response JSON string part
output = generator_output_str.split("=")
print(output[2] + output[3])
# len(output)


'Benzoyl peroxide topical is used to treat mild to moderate acne, reduce inflammation and kill bacteria that cause acne.', 
        example 'For example, it may be applied directly to the skin to help clear up pimples and reduce redness and inflammation in the affected area.'
    ), 
    error
